# Problema das Garrafas

O Problema de Otimização da Fabricação de Garrafas Plásticas é um simples "problema brinquedo" utilizado para introduzir, na prática, a aplicação de Algoritmos Genéticos em problemas de otimização.

Para mais detalhes, a apresentação do problema se encontra [aqui](http://silverio.net.br/heitor/disciplinas/ce/arquivos/garrafas2020.pdf). A solução é apresentada neste [documento](http://silverio.net.br/heitor/disciplinas/ce/arquivos/garrafas2020-solucao.pdf).

In [1]:
import numpy as np
from deap import base, creator, tools

# Python complains about overwriting the Individual class. Since that is intended I have decided to hide the warning
import warnings
warnings.filterwarnings('ignore')

# Set seed for reproducibility
np.random.seed(42)

Declaração da função de avaliação, responsável por calcular o fitness de um indivíduo. As funções de penalização, como apresentadas na solução do problema, são declaradas logo em seguida.

In [2]:
# Calculates fitness for a given individual
def evaluate(ind):
    obj = (5 * ind[0]) + (4.5 * ind[1])
    obj = obj / 6650
    pen = h1(ind) + h2(ind) + h3(ind) + h4(ind)
    pen = pen / 2
    return (obj - pen),

# Declaration of penalization functions as per the solution provided

def h1(ind):
    return max(0, (6 * ind[0] + 5 * ind[1])/100 - 60) / 60

def h2(ind):
    return max(0, (10 * ind[0]) + (20 * ind[1]) - 14000) / 14000

def h3(ind):
    return max(0, (ind[0] - 700)) / 700

def h4(ind):
    return max(0, (ind[1] - 700)) / 700

Definição de parâmetros:

In [3]:
pop_size = 300
generations = 1000
# Qty of variables/genes
ind_size = 2
# Mutation probability
mutpb = 0.2
# Crossover probability
cxpb = 0.5

Criação dos tipos que serão utilizados. Aqui define-se o "esqueleto" da função de Fitness e dos indivíduos.

In [4]:
# Create types
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

`toolbox` contém todos os objetos que serão utilizados (indivíduo, população, funções, operadores e argumentos) e, portanto, devem ser declarados aqui.

In [5]:
toolbox = base.Toolbox()
# Random attibute generator. Used for random initialization of individuals. Ranges from 0 to 1024
toolbox.register("attribute", np.random.randint, 1025)
# Definition of the individual. n defines the number of genes
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attribute, n=ind_size)
# Definition of the population
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Genetic operators
toolbox.register("evaluate", evaluate)
# Two-point crossover
toolbox.register("mate", tools.cxOnePoint)
# Mutation by flipping bits
toolbox.register("mutate", tools.mutUniformInt, low=0, up=700, indpb=0.05)
# Tournament selection
toolbox.register("select", tools.selTournament, tournsize=3)

Função principal onde acontece o processo de evolução. O critério de parada é o número da geração atual é quando `g` é igual à `generations`

In [6]:
def main():
    pop = toolbox.population(n=pop_size)
    
    # Evaluate population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    
    # Keeps track of the number of generations
    g = 0
    
    fits = [ind.fitness.values[0] for ind in pop]
    
    # Begin the evolution    
    while g < generations:
        g = g + 1
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
        
        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if np.random.rand() < cxpb:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if np.random.rand() < mutpb:
                toolbox.mutate(mutant)
                del mutant.fitness.values
                
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        pop[:] = offspring

        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        if g % 100 == 0 or g == 1:
            print(f"-- Generation {g} --")
            print("  Min %s" % min(fits))
            print("  Max %s" % max(fits))
            print("  Avg %s" % mean)
            print("  Std %s" % std)
        
    print("-- End of evolution process --")
    
    best_ind = tools.selBest(pop, 1)[0]

    print()
    print(f"Qtd de garrafas de leite: {best_ind[0]}")
    print(f"Qtd de garrafas de suco: {best_ind[1]}")
    print(f"Ocupação da extrusora: {(6 * best_ind[0]/100) + (5 * best_ind[1]/100)}h")
    print(f"Ocupação do depósito: {10 * best_ind[0] + 20 * best_ind[1]}")
    print(f"Lucro: {5 * best_ind[0] + 4.5 * best_ind[1]}")

In [7]:
if __name__ == "__main__":
    main()

-- Generation 1 --
  Min 0.11323308270676692
  Max 0.7571992481203007
  Avg 0.595999956140351
  Std 0.10686896171550742
-- Generation 100 --
  Min 0.5451127819548872
  Max 0.7625187969924813
  Avg 0.7608665413533833
  Std 0.01780756582518037
-- Generation 200 --
  Min 0.26917293233082706
  Max 0.7625187969924813
  Avg 0.757311931913116
  Std 0.04084312798197915
-- Generation 300 --
  Min 0.2586466165413534
  Max 0.7625187969924813
  Avg 0.7570104803675854
  Std 0.04347656708874782
-- Generation 400 --
  Min 0.281203007518797
  Max 0.7631578947368421
  Avg 0.7532721908939027
  Std 0.05175699256817248
-- Generation 500 --
  Min 0.24511278195488723
  Max 0.7631578947368421
  Avg 0.7592277360066849
  Std 0.038636882909161646
-- Generation 600 --
  Min 0.27225563909774436
  Max 0.7631578947368421
  Avg 0.7600206662489571
  Std 0.031750507576047866
-- Generation 700 --
  Min 0.44360902255639095
  Max 0.7631578947368421
  Avg 0.761080816624897
  Std 0.022940182402279633
-- Generation 800 --
 